In [60]:
import os, shutil

base_dir = '/Volumes/KESU/dog_vs_cat'
dataset = os.path.join(base_dir, 'dataset')
os.mkdir(dataset)

train_dir = os.path.join(dataset, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(dataset, 'validation')
os.mkdir(validation_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

In [61]:
#trainデータの８割を学習用、残りをバリデーション用として使用する
#total number of images 12500 
#train images 10000
#validation images 2500

fnames_cat = ['cat.{}.jpg'.format(i) for i in range(12500)]
fnames_dog = ['dog.{}.jpg'.format(i) for i in range(12500)]

In [62]:
#重複を許さず、猫ファイル名を取得
import random

# 乱数により移動先を振り分け
train = os.path.join(base_dir, 'train')

for cat in fnames_cat:
    src = os.path.join(train, cat)
    if random.random() >= 0.2: 
        # 学習用フォルダへ移動
        dst = os.path.join(train_cats_dir,cat)
        shutil.copy(src,dst)
    else:                      
        dst = os.path.join(validation_cats_dir,cat)
        shutil.copy(src,dst)

for dog in fnames_dog:
    src = os.path.join(train, dog)
    if random.random() >= 0.2: 
        # 学習用フォルダへ移動
        dst = os.path.join(train_dogs_dir,dog)
        shutil.copy(src,dst)
    else:                      
        # dir2へ移動
        dst = os.path.join(validation_dogs_dir,dog)
        shutil.copy(src,dst)

In [63]:
from keras.models import Model

from keras.layers import Input, Activation, merge, Dense, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D

from keras.optimizers import Adam

from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import toimage

In [64]:
def _shortcut(inputs, residual):
  n_filters = residual._keras_shape[3]
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='he_normal', padding='valid')(inputs)
                           
  return add([shortcut, residual])

In [65]:
def _resblock(n_filters, strides=(1,1)):
    def f(input):
        x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(input)
        x = BatchNormalization()(x)
        return _shortcut(input, x)

    return f

In [90]:
def resnet():
  inputs = Input(shape=(150, 150, 3))
  x = Convolution2D(32, (7,7), strides=(1,1),kernel_initializer='he_normal', padding='same')(inputs)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  
  x = MaxPooling2D((3, 3), strides=(2,2), padding='same')(x)
  
  
  x = _resblock(n_filters=64)(x)
  x = _resblock(n_filters=64)(x)
  x = _resblock(n_filters=64)(x)
  x = MaxPooling2D(strides=(2,2))(x)  
  x = _resblock(n_filters=128)(x)
  x = _resblock(n_filters=128)(x) 
  x = _resblock(n_filters=128)(x)

  x = GlobalAveragePooling2D()(x)
  x = Dense(1, kernel_initializer='he_normal', activation='softmax')(x)

  model = Model(inputs=inputs, outputs=x)
  return model

In [91]:
model = resnet()

adam = Adam()

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_117 (Conv2D)             (None, 150, 150, 32) 4736        input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_81 (BatchNo (None, 150, 150, 32) 128         conv2d_117[0][0]                 
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 150, 150, 32) 0           batch_normalization_81[0][0]     
__________________________________________________________________________________________________
max_poolin

In [80]:
import os, shutil

base_dir = '/Volumes/KESU/dog_vs_cat'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [81]:
train = os.path.join(dataset,'train')
validation = os.path.join(dataset,'validation')

In [82]:
dataset = os.path.join(base_dir, 'dataset')
#os.mkdir(os.path.join(dataset, 'train'))
#os.mkdir(os.path.join(dataset, 'validation'))

In [83]:
print('total training  images', len(os.listdir(train)))
print('///////////////////////////////////////////////////')
print('total validation  images', len(os.listdir(validation)))
print('total test images', len(os.listdir(validation)))

total training  images 3
///////////////////////////////////////////////////
total validation  images 3
total test images 3


In [84]:
#preprocessing
#画像ファイルを読み込む
#RGBのピクセルグリッドにデコードする
#テンソルに変換する
#[0, 1]に収まるようにする

from keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(rescale=1./255)
val_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

train_generator = train_data_generator.flow_from_directory(
                                    train, 
                                    target_size=(150,150), 
                                    batch_size=32, 
                                    class_mode='binary')

val_generator = val_data_generator.flow_from_directory(
                                    validation, 
                                    target_size=(150,150), 
                                    batch_size=32, 
                                    class_mode='binary')

test_generator = test_data_generator.flow_from_directory(
                                    test_dir, 
                                    target_size=(150,150), 
                                    batch_size=32, 
                                    class_mode='binary')

Found 20133 images belonging to 2 classes.
Found 4867 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [85]:
from keras import optimizers

model.compile(loss='binary_crossentropy', 
                             optimizer= optimizers.RMSprop(lr=1e-3), 
                             metrics= ['acc'])

In [86]:
for data_batch, labels_batch in train_generator:
    print('data batch shape', data_batch.shape)
    print('/////////////////////////////////')
    print('labels batch size', labels_batch.shape)
    break

data batch shape (32, 150, 150, 3)
/////////////////////////////////
labels batch size (32,)


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

callback_cp = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=2, mode='auto', verbose=1)

history = model.fit_generator(train_generator, 
                                                     steps_per_epoch=30, 
                                                     epochs=30, 
                                                     validation_data=val_generator, 
                                                     callbacks=[callback_cp, callback_es],
                                                     validation_steps=50)

Epoch 1/30
